In [ ]:
# 安装aix360
!pip install git+https://github.com/Trusted-AI/AIX360.git

In [ ]:
# 导入包
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import xgboost
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5))   #returns False, without Warning
from sklearn.model_selection import train_test_split
from aix360.algorithms.protodash import ProtodashExplainer
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 导入数据并区分X和y
heloc = pd.read_csv('../input/home-equity-line-of-creditheloc/heloc_dataset_v1 (1).csv')
X = heloc.drop(columns = 'RiskPerformance')
y = heloc.RiskPerformance.replace(to_replace=['Bad', 'Good'], value=[1, 0])

In [ ]:
# 切分训练集及测试集并转换成array
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)

In [ ]:
# 基于XGBOOST模型构建解释器
model = xgboost.XGBClassifier().fit(X_train_array, y_train_array)

In [ ]:
# 计算模型预测准确率
pred_train = model.predict(X_train_array)
pred_test = model.predict(X_test_array)
score_train = sklearn.metrics.accuracy_score(y_train, pred_train)
score_test = sklearn.metrics.accuracy_score(y_test,pred_test)
print("train accuracy:",score_train)
print("test accuracy:",score_test)

In [ ]:
def protodash_explain(X_train,X_test,model,idx,m):
    # 将训练数据集与模型的预测值合并成同一个array，并区分预测结果分别为好/坏的样本
    X_train_array = np.array(X_train)
    X_test_array = np.array(X_test)
    pred_train = model.predict(X_train_array)
    pred_train = pred_train.reshape((pred_train.shape[0],1)) 
    z_train = np.hstack((X_train_array, pred_train)) 
    z_train_good = z_train[z_train[:,-1]==0, :]
    z_train_bad = z_train[z_train[:,-1]==1, :]
    # 从测试数据集中随机选取待解释样本
    print("Chosen Sample:", idx)
    print("Prediction made by the model:", model.predict(X_test_array)[idx])
    print("Prediction probabilities:", model.predict_proba(X_test_array[idx].reshape(1,X_test_array.shape[1])))
    print("")
    Target = np.hstack((X_test_array[idx].reshape((1,23)), model.predict(X_test_array)[idx].reshape((1,1))))
    print("待解释样本特征取值如下：")
    print(X_test.iloc[idx])
    # 创建Protodash解释器
    explainer = ProtodashExplainer()
    (W, S, setValues) = explainer.explain(Target, z_train_good, m) 
    # 展示各Prototype特征取值以及相应的Weight
    col = X_train.columns.tolist()
    col.append('RiskPerformance')
    if model.predict(X_test_array)[idx]==0:
        prototypes = pd.DataFrame(z_train_good[S, :])
    else:
        prototypes = pd.DataFrame(z_train_bad[S, :])
    prototypes.columns = col
    prototypes["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5))
    prototypes = prototypes.transpose()
    # 展示各Prototype的各个特征的Weight
    if model.predict(X_test_array)[idx]==0:
        z = z_train_good[S, 0:-1] # Store chosen prototypes
    else:
        z = z_train_bad[S, 0:-1] # Store chosen prototypes
    eps = 1e-10 # Small constant defined to eliminate divide-by-zero errors
    feature_weight = np.zeros(z.shape)
    for i in range (z.shape[0]):
        for j in range(z.shape[1]):
            feature_weight[i, j] = np.exp(-1 * abs(Target[0, j] - z[i,j])/(np.std(z[:, j])+eps)) # Compute feature similarity in [0,1]
    # move wts to a dataframe to display
    feature_weight_table = pd.DataFrame.from_records(np.around(feature_weight.astype('double'), 2))
    feature_weight_table.columns = X_train.columns.tolist()
    feature_weight_table = feature_weight_table.transpose() 
    return prototypes,feature_weight_table


In [ ]:
protodash_explain(X_train=X_train,X_test=X_test,model=model,idx=8,m=5)

In [ ]:
# 将训练数据集与模型的预测值合并成同一个array，并区分预测结果分别为好/坏的样本
pred_train = pred_train.reshape((pred_train.shape[0],1)) 
z_train = np.hstack((X_train_array, pred_train)) 
z_train_good = z_train[z_train[:,-1]==0, :]
z_train_bad = z_train[z_train[:,-1]==1, :]

In [ ]:
# 从测试数据集中随机选取待解释样本
idx = 9
print("Chosen Sample:", idx)
print("Prediction made by the model:", model.predict(X_test_array)[idx])
print("Prediction probabilities:", model.predict_proba(X_test_array[idx].reshape(1,X_test_array.shape[1])))
print("")
Target = np.hstack((X_test_array[idx].reshape((1,23)), model.predict(X_test_array)[idx].reshape((1,1))))
print("待解释样本特征取值如下：")
X_test.iloc[idx]

In [ ]:
# 创建Protodash解释器
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(Target, z_train_good, m=5) 

In [ ]:
col = X_train.columns.tolist()
col

In [ ]:
col.append('RiskPerformance')

In [ ]:
col

In [ ]:
# 展示各Prototype特征取值以及相应的Weight
prototypes = pd.DataFrame(z_train_good[S, :])
prototypes.columns = col
prototypes["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5))
prototypes.transpose()

In [ ]:
# 展示各Prototype的各个特征的Weight
z = z_train_good[S, 0:-1] # Store chosen prototypes
eps = 1e-10 # Small constant defined to eliminate divide-by-zero errors
feature_weight = np.zeros(z.shape)
for i in range (z.shape[0]):
    for j in range(z.shape[1]):
        feature_weight[i, j] = np.exp(-1 * abs(Target[0, j] - z[i,j])/(np.std(z[:, j])+eps)) # Compute feature similarity in [0,1]
                
# move wts to a dataframe to display
feature_weight_table = pd.DataFrame.from_records(np.around(feature_weight.astype('double'), 2))
feature_weight_table.columns = X_train.columns.tolist()
feature_weight_table.transpose() 